In [22]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch import optim

import torchvision 
import torchvision.transforms as transforms

#Device_config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyperparemeters (#to be tuned)
input_size = 784 #28x28
hidden_size = 100
num_classes = 10
num_epochs = 6
batch_size = 100
learning_rate = 0.003 

In [ ]:
#Import Data
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = False,
                                           transform = transforms.ToTensor(),
                                           download = True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle=False)

In [24]:
#Exemples
examples = iter(train_loader)
samples, labels = examples.next()
#print(samples.shape, labels.shape)

"""for i in range(6):
    plt.subplot(2, 3, i+1) #2 rows, 3 columns
    plt.imshow(samples[i][0], cmap='gray') #sample i in channel 0 (Cuz we have only 1 channel)
plt.show()"""

"for i in range(6):\n    plt.subplot(2, 3, i+1) #2 rows, 3 columns\n    plt.imshow(samples[i][0], cmap='gray') #sample i in channel 0 (Cuz we have only 1 channel)\nplt.show()"

In [25]:
#NN Class
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__() #super ?
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

#Loss & Optimizer
criterion = nn.CrossEntropyLoss() #This will implement the softmax for us
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#Training_loop
n_tot_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print(f'epoch = {epoch+1}/{num_epochs}, step = {i+1}/{n_tot_steps}, loss = {loss.item():.4f}')

epoch = 1/6, step = 100/600, loss = 0.5452
epoch = 1/6, step = 200/600, loss = 0.2968
epoch = 1/6, step = 300/600, loss = 0.4820
epoch = 1/6, step = 400/600, loss = 0.0980
epoch = 1/6, step = 500/600, loss = 0.1967
epoch = 1/6, step = 600/600, loss = 0.2096
epoch = 2/6, step = 100/600, loss = 0.0691
epoch = 2/6, step = 200/600, loss = 0.1584
epoch = 2/6, step = 300/600, loss = 0.1005
epoch = 2/6, step = 400/600, loss = 0.0890
epoch = 2/6, step = 500/600, loss = 0.0829
epoch = 2/6, step = 600/600, loss = 0.1086
epoch = 3/6, step = 100/600, loss = 0.0828
epoch = 3/6, step = 200/600, loss = 0.0531
epoch = 3/6, step = 300/600, loss = 0.0708
epoch = 3/6, step = 400/600, loss = 0.2243
epoch = 3/6, step = 500/600, loss = 0.1166
epoch = 3/6, step = 600/600, loss = 0.0743
epoch = 4/6, step = 100/600, loss = 0.0750
epoch = 4/6, step = 200/600, loss = 0.0124
epoch = 4/6, step = 300/600, loss = 0.0423
epoch = 4/6, step = 400/600, loss = 0.0314
epoch = 4/6, step = 500/600, loss = 0.1188
epoch = 4/6

In [26]:
#test
with torch.no_grad(): #we don't want to calculate gradient for test set
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device) #why the reshape ?
        labels = labels.to(device)
        outputs = model(images)
        
        #value, index
        _, predictions = torch.max(outputs, 1) #torch.max ?
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100.0 * n_correct/n_samples
    print(f'accuracy = {acc}')

accuracy = 97.61


In [28]:
#Prediction on one test loader image

testo = iter(test_loader)
s , l = testo.next()

with torch.no_grad():
    image = s[2].reshape(-1, 28*28).to(device) #prediction of the label of the third image [2] in the test_dataset   
    label = l[2].to(device)
    outp = model(image)
    _ , pred = torch.max(outp,1)
    print(f'pred = {pred.item()} & label = {label}')

pred = 1 & label = 1
